In [1]:
from multion.client import MultiOn

multion_client = MultiOn()


In [29]:
def session(url: str = "https://google.com", local: bool = False):
    s = multion_client.sessions.create(url=url, local=local)
    return s.session_id


def session_step(session_id: str, cmd: str):
    return multion_client.sessions.step(session_id=session_id, cmd=cmd)


def session_step_run(session_id: str, cmd: str):
    status = "CONTINUE"
    while status == "CONTINUE":
        status = session_step(session_id=session_id, cmd=cmd).status
        print(status)

        message = session_step(session_id=session_id, cmd=cmd).message
        if message:
            print(message)
    return message


In [32]:
session_id = session(url="https://yande.re/post", local=False)

session_step_run(session_id, "Get an image of komeiji_koishi and get its url.")


CONTINUE
I am clicking on the first image of "komeiji_koishi" to view it and get its URL.

CONTINUE
I am clicking on the "Download larger version" link to get the high-quality version of the image and obtain its URL.

DONE
The URL of the image of "komeiji_koishi" is:
https://files.yande.re/image/24a43b7aa4f5e405d7f6b1b6632dd72a/yande.re%201186002%20bikini%20cameltoe%20dress%20ke-ta%20komeiji_koishi%20loli%20skirt_lift%20summer_dress%20swimsuits%20touhou.jpg



In [26]:
def session_get(
    cmd: str,
    fields: list[str],
    url: str = "https://google.com",
    local: bool = True,
    max_items: int = 1,
    render_js: bool = True,
):
    out = multion_client.retrieve(
        cmd=cmd,
        fields=fields,
        url=url,
        local=local,
        max_items=max_items,
        render_js=render_js,
    )
    return out.data


In [27]:
dt = session_get(
    cmd="Get an image of komeiji_koishi.",
    fields=["image"],
    url="https://yande.re/post",
)
dt


In [35]:
from pydantic import BaseModel
import openai

_llm_client = openai.OpenAI()


class MultiOnMessage(BaseModel):
    command: str
    url: str | None


def multion_message(prompt: str):
    c = _llm_client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """You are an expert at structured data extraction.
You will be given unstructured text from a research paper and should convert it into the given structure.

Example input: 从 pixiv.com 找一张古明地恋的图片.
Example output: {"command": "找一张古明地恋的图片.", "url": "https://pixiv.com"}
                """,
            },
            {"role": "user", "content": prompt},
        ],
        response_format=MultiOnMessage,
    )
    multion_message = c.choices[0].message
    if multion_message.refusal:
        print(f"Refusal: {multion_message.refusal}")
    else:
        return multion_message.parsed


In [38]:
msg = multion_message("Get an image of komeiji_koishi from yande.re/post.")


In [39]:
msg.command


'Get an image of komeiji_koishi.'